In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import Imputer
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# WIDS 2018 Datathon

The dataset for the challenge will contain demographic and behavioral information from a representative sample of survey respondents from India and their usage of traditional and mobile financial services. The dataset is a product of InterMedia’s research to help the world’s poorest people take advantage of widely available mobile phones and other digital technology to access financial tools and participate more fully in their local economies. Women in these communities, in particular, are often largely excluded from the formal financial system. By predicting gender, the datathon teams will explore the key differences in behavior patterns of men and women, and how that may impact their use of new financial services. Ideally, these findings will influence plans to reach women in developing economies and encourage them to adopt new financial tools that will help to lift them and their families out of poverty.

## Goal
For each test_id in the test set, you must predict a probability for the person being female

## Data
The training set has 18255 entries with 1235 features.
The test set has 27285 entries with 1234 features.
Both datasets have the same columns except for 
* test_id and train_id
* the additional column is_female in the training set (which is our target value)

In [ ]:
test = pd.read_csv('../input/test.csv', low_memory=False)
train = pd.read_csv('../input/train.csv', low_memory=False)

In [ ]:
print("Test size", test.shape)
print("Train size", train.shape)
test_headers = test.columns.values.tolist()
train_headers = train.columns.values.tolist()
combined = train.append(test)
print("Combined size", combined.shape)
print("Column differences", np.setdiff1d(train_headers, test_headers), np.setdiff1d(test_headers,train_headers))

## Preprocessing
First, we will take a look at the train and test data. If we look at the data dictionary excel file, we can see that all columns are of categorical (or at least integer) nature and that most of them are encoded answers to interview questions. 

To preprocess the large amount of data in a meaningful way, we first drop columns with more than 75% missing data and the columns which most likely describe interview circumstances.

In [ ]:
X_train = train
X_test = test
y_train = train.is_female

X_train.drop(['AA4', 'AA5', 'AA6', 'AA7', 'AA14', 'AA15'], axis=1, inplace=True)
X_train.dropna(thresh=0.75*len(X_train), axis=1, inplace=True)
X_train.drop(['train_id', 'is_female'], axis=1, inplace=True)

print("Remaining columns:", np.size(X_train.columns.values.tolist()))

X_test = X_test[X_train.columns.values.tolist()]


We can see, for example, that columns encoding answers for "OTHERS" were dropped. Out of the remaining columns, some are still missing values.

In [ ]:
col_indices_nan = np.array(np.where(X_train.isnull().sum()>0))[0] # indices of columns with missing values
print("Number of columns with missing values", np.size(col_indices_nan)) 
X_headers_nan = [X_train.columns.values.tolist()[i] for i in col_indices_nan] # labels of columns with missing values

Before we start imputing the missing values, we check the nature of the remaining columns. All columns have integer values which could be interpreted as both nominal (e.g. Yes/No) or ordinal data ("On a scale from 1 to 4..."). Every categorical column can also have a different number of possible values. 

Columns with many unique values have a high probability of being ordinal variables.
* DG1: Year of birth **(ordinal)**
* FB13: How many times in the past 12 months have you borrowed money (from outside your household)? **(ordinal)**
* DL14: Unknown
* FL4: What or who do you depend on the most for financial advice? (nominal)
* DL11: In the past 12 months,how many times did you move from one home to another? **(ordinal)**
* FB20: What is the main reason you do not borrow from a bank? (nominal)
* MT1: How many people in your household have a mobile phone? **(ordinal)**
* DG8a - DG8c, DG9a - DG9c: How many... **(ordinal)**
* FL10: What’s the most important financial goal for you right now? (nominal)
* DG4: What is your highest level of education? **(ordinal or nominal)**
* FL9A: Imagine that this month, after paying for food, cooking fuel, school fees, rent, and airtime, you found yourself with some extra money. Please, select 3 options from the list that you are most likely to spend it on, Option 1. (nominal)
* DL1: In the past 12 months, were you mainly...? (nominal)
* FL9B: Imagine that this month, after paying for food, cooking fuel, school fees, rent, and airtime, you found yourself with some extra money. Please, select 3 options from the list that you are most likely to spend it on, Option 2. (nominal)
* FB19: Which of the following best describes how you spent the money you borrowed last time? (nominal)
* IFI16_1: If you want to get to Over the counter in a branch of a bank , how would you get there? Would you…? (nominal)
* DG3: What is your marital status? (nominal)
* DG6: How are you related to the household head? (nominal)
* IFI18: How many informal societies or group saving schemes do you use or personally belong to? **(ordinal)**
* DG3A: What is your religion? (nominal)
* IFI14_1 - IFI14_7: How close... **(ordinal)**
* IFI15_1 - IFI15_7: How much time... **(ordinal)**
* MT1A: Who decides on who should have a phone in your household? (nominal)
* DL24: Please look at this card and tell me which answer best reflects your family's financial situation? (nominal)
* IFI17_1 - IFI17-7: On a scale... **(ordinal)**
* GN1 - GN5: Who decides... (nominal)
* FL8_1 - FL8_7: How much do you agree with ... **(ordinal)**
* FL11: How likely is it that you could get together sufficient funds from your friends and family for a medical emergency if it was too much for you to manage alone? **(ordinal)**
* FB18: Which of the following statements best describes how you usually repay your loans? (nominal)
* LN2_1 - LN2_4: On a scale... **(ordinal)**
* FL8_1 - FL8_7: How much do you agree... **(ordinal)**
* DL15: What is the highest grade that the female head/spouse completed? **(ordinal or nominal)**
* FL1: How often do you make a plan for how to manage your money, whether it is earned through a job, received from the government or from other people? **(ordinal)**
* FL15: Suppose over the next 10 years the prices of the things you buy double. If your income also doubles, will you be able to buy less than you can buy today, the same as you can buy today, or more than you can buy today? (nominal, it is a quiz answer)
* AA3: Zone (nominal)
* LN1A, LN1B: Literacy **(ordinal)**
* All other columns have less than or equal to 3 unique values, which we will take as being nominal

Ordinal columns will not be one-hot encoded later on. This means that we need to deal with the placeholder value 99 which is used to encode the answering option "Don't know". Otherwise the value distorts ordinal relationships. We will therefore replace both missing values and the value 99 with the median value in ordinal columns.

In [ ]:
print("Columns with the most unique values")
print(X_train.apply(pd.Series.nunique).sort_values(axis=0,ascending=False)[0:10])

Column DG1 represents birth years. We will create a new age column which assigns each sample to an age bin.

In [ ]:
class BinCreator:
    def __init__(self,colSeries):
        self.quantiles=colSeries.quantile([.25, .5, .75]).values
    def assignBin(self, x):
        if x <= self.quantiles[0]:
            return 1
        if x <= self.quantiles[1]:
            return 2
        if x <= self.quantiles[2]:
            return 3
        else:
            return 4

X_test.DG1.head()

In [ ]:
# apply agebin transformation to training and test set
ageBinCreator = BinCreator(X_train.DG1)
X_train.loc[:,'agebin'] = np.vectorize(ageBinCreator.assignBin)(X_train['DG1'])
X_test.loc[:,'agebin'] = np.vectorize(ageBinCreator.assignBin)(X_test['DG1'])

# drop the original DG1 column
X_train = X_train.drop(['DG1'], axis=1) 
X_test = X_test.drop(['DG1'], axis=1)
X_test.agebin.head()

For the other ordinal columns, we will replace the placeholder value 99 with NaN first.

In [ ]:
ordinal = [
    'agebin', 'MT1bin', 'DG8abin', 'DG8bbin', 'DG8cbin', 'DG9abin', 'FB13', 'DL14', 'DL11'
    'FL8_1', 'FL8_2','FL8_3','FL8_4','FL8_5','FL8_6','FL8_7',
    'LN2_1','LN2_2','LN2_3','LN2_4',
    'MM42_1', 'MM42_2', 'MM42_3', 'MM42_4', 'MM42_5', 'MM42_6',
    'MM32_1', 'MM32_2', 'MM32_3', 'MM32_4', 'MM32_5', 'MM32_6', 'MM32_7', 'MM32_8', 'MM32_9', 'MM32_10','MM32_11', 'MM32_12', 'MM32_13', 
    'IFI17_1', 'IFI17_2', 'IFI17_3', 'IFI17_4', 'IFI17_5', 'IFI17_6', 'IFI17_7',
    'LN1B', 'LN1A',
    'RI6_1', 'RI6_2', 'RI6_3', 'RI7_1', 'RI7_2', 'RI7_3',
    'FB13',
    'DL14',
    'DL11',
    'MT1',
    'DG8a', 'DG8b', 'DG8c', 'DG9a', 'DG9b', 'DG9c',
    'IFI18'
]
intersect = np.intersect1d(ordinal,X_train.columns.values.tolist(),assume_unique=True)
X_train.loc[:,intersect] = X_train[intersect].replace(to_replace=[99],value=[np.nan],inplace=False)

# do the same for test set
X_test.loc[:,intersect] = X_test[intersect].replace(to_replace=[99],value=[np.nan],inplace=False)    

In [ ]:
X_train.dropna(thresh=0.75*len(X_train), axis=1, inplace=True)
print("Remaining columns:",np.size(X_train.columns.values.tolist()))

X_test=X_test[X_train.columns.values.tolist()]

After replacing all 99 (Don't know) with NaN in both train and test sets and dropping all resulting columns with more than 25% missing values, we can impute the missing values for the ordinal data.

In [ ]:
intersect = np.intersect1d(ordinal,X_train.columns.values.tolist(),assume_unique=True)

imp = Imputer(missing_values='NaN', strategy='median', axis=0)
imp.fit(X_train[intersect])

# train set
X_train[intersect]=imp.transform(X_train[intersect])

# test set
X_test[intersect] = imp.transform(X_test[intersect])

Next we impute the missing nominal values with the column mode. We can leave 99 as its own category.

In [ ]:
nominal = np.setdiff1d(X_train.columns.values.tolist(), ordinal)

imp_nom = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
imp_nom.fit(X_train[nominal])

# train set
X_train[nominal] = imp_nom.transform(X_train[nominal])
X_train.head()

# test set
X_test[nominal] = imp_nom.transform(X_test[nominal])

Now we can convert all column types to integer

In [ ]:
#convert float to int
numeric = X_train.select_dtypes(include=np.number).columns
non_numeric = train.select_dtypes(exclude=np.number).columns # _OTHERS columns
print("Numeric columns", np.size(numeric))
print("Other columns", np.size(non_numeric))

X_train[numeric]=X_train[numeric].astype(int)
X_test[numeric]=X_test[numeric].astype(int)

print(np.shape(X_train))
print(np.shape(X_test))
X_train.head()

We have created age bins based on age quantiles of the training set. The columns FB13, DL14, DL11, MT1, DG8a- DG8c, DG9a - DG9c and IFI18 (DG9b, DG9c, IFI18 were dropped already) still have many levels (>12) and are not binned by default. We will group them as well.

In [ ]:
FB13BinCreator = BinCreator(X_train.FB13)
X_train.loc[:,'FB13bin'] = np.vectorize(ageBinCreator.assignBin)(X_train['FB13'])
X_test.loc[:,'FB13bin'] = np.vectorize(ageBinCreator.assignBin)(X_test['FB13'])

FB13BinCreator = BinCreator(X_train.FB13)
X_train.loc[:,'DL14bin'] = np.vectorize(ageBinCreator.assignBin)(X_train['DL14'])
X_test.loc[:,'DL14bin'] = np.vectorize(ageBinCreator.assignBin)(X_test['DL14'])

FB13BinCreator = BinCreator(X_train.FB13)
X_train.loc[:,'DL11bin'] = np.vectorize(ageBinCreator.assignBin)(X_train['DL11'])
X_test.loc[:,'DL11bin'] = np.vectorize(ageBinCreator.assignBin)(X_test['DL11'])

FB13BinCreator = BinCreator(X_train.FB13)
X_train.loc[:,'MT1bin'] = np.vectorize(ageBinCreator.assignBin)(X_train['MT1'])
X_test.loc[:,'MT1bin'] = np.vectorize(ageBinCreator.assignBin)(X_test['MT1'])

FB13BinCreator = BinCreator(X_train.FB13)
X_train.loc[:,'DG8abin'] = np.vectorize(ageBinCreator.assignBin)(X_train['DG8a'])
X_test.loc[:,'DG8abin'] = np.vectorize(ageBinCreator.assignBin)(X_test['DG8a'])

FB13BinCreator = BinCreator(X_train.FB13)
X_train.loc[:,'DG8bbin'] = np.vectorize(ageBinCreator.assignBin)(X_train['DG8b'])
X_test.loc[:,'DG8bbin'] = np.vectorize(ageBinCreator.assignBin)(X_test['DG8b'])

FB13BinCreator = BinCreator(X_train.FB13)
X_train.loc[:,'DG8cbin'] = np.vectorize(ageBinCreator.assignBin)(X_train['DG8c'])
X_test.loc[:,'DG8cbin'] = np.vectorize(ageBinCreator.assignBin)(X_test['DG8c'])

FB13BinCreator = BinCreator(X_train.FB13)
X_train.loc[:,'DG9abin'] = np.vectorize(ageBinCreator.assignBin)(X_train['DG9a'])
X_test.loc[:,'DG9abin'] = np.vectorize(ageBinCreator.assignBin)(X_test['DG9a'])



In [ ]:
X_train = X_train.drop(['FB13'], axis=1) 
X_test = X_test.drop(['FB13'], axis=1)
X_train = X_train.drop(['DL14'], axis=1) 
X_test = X_test.drop(['DL14'], axis=1)
X_train = X_train.drop(['DL11'], axis=1) 
X_test = X_test.drop(['DL11'], axis=1)
X_train = X_train.drop(['MT1'], axis=1) 
X_test = X_test.drop(['MT1'], axis=1)
X_train = X_train.drop(['DG8a'], axis=1) 
X_test = X_test.drop(['DG8a'], axis=1)
X_train = X_train.drop(['DG8b'], axis=1) 
X_test = X_test.drop(['DG8b'], axis=1)
X_train = X_train.drop(['DG8c'], axis=1) 
X_test = X_test.drop(['DG8c'], axis=1)

We one-hot encode the nominal columns. A custom encoder needs to be written because scikit-learn's standard one-hot encoder turns all integers between [min(column), max(column)] into categories. In our dataset however, the number of categories (=answering options) is mostly lower than 10, but the integer value 99 acts as placeholder for the answering option "Don't know".

In [ ]:
class CustomOneHot(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns = []

    def transform(self, X):
        encoded = pd.get_dummies(X, columns=self.columns, prefix=self.columns)
        return encoded

    def fit(self, X):
        self.columns = X.columns.values.tolist()

X_train.head()
nominal = np.setdiff1d(X_train.columns.values.tolist(), ordinal)
onehot = CustomOneHot()
onehot.fit(X_train[nominal])

In [ ]:
X_train = onehot.transform(X_train)
X_test = onehot.transform(X_test)
X_train.head()

## Feature Selection

Based on chi squared test, we choose 140 features with the strongest correlation to the target variable

In [ ]:
transformA=SelectKBest(chi2, k=140).fit(X_train, y_train)
supportA_index = np.array(np.where(transformA.get_support()==True))[0]
labelsA = [X_train.columns.values.tolist()[i] for i in supportA_index]
labels = labelsA

## Modelling

The goal of this project is to get familiar with different models and then to combine suitable models in an ensemble.

### Logistic Regression
We can now train several models to see how well a single model can perform. We start with logistic regression and search for the best regularization parameter C with LogisticRegressionCV which is faster than GridSearchCV.

In [ ]:
log_clf = LogisticRegressionCV(Cs=[0.01, 0.03, 0.1, 0.3, 1, 3], scoring='roc_auc', cv=3, penalty='l1', random_state=0, refit=True, solver='liblinear')
log_clf.fit(X_train[labels], y_train)
log_clf.scores_, log_clf.C_ # best: C=0.3

In [ ]:
result_log = log_clf.predict_proba(X_test[labels])[:,1]
#pd.DataFrame(result_log).to_csv('results_log.csv',index=True, index_label='test_id', header = ["is_female"])

Increasing the number of features does not seem to increase the score much. The validation score is only 95.9, but the difference between validation and training score is below 0.5, which means that this model doesn't have an overfitting problem.

### Support Vector Machines
It turns out that SVM performance is very bad with a dataset of such high dimensions.
LinearSVR (SVM with a linear kernel) is very inaccurate.

### Gradient Boosted Trees
Gradient Boosted Trees with xgboost is the most performant. We notice that without any data preprocessing, the algorithm can reach a score of about 97% (see `wids-xgboost.ipynb`). With preprocessing, xgboost does not perform better. This could hint to suboptimal preprocessing such as the handling of missing errors or the encoding of categorical data. Scikit-learn's GradientBoostingClassifier performs equally well, but needs more computation time.

In [ ]:
# GBT with scikit-learn
param_gb = {
    'n_estimators': [200], 
    'learning_rate': [0.1], 
    'max_depth': [5, 6, 7],
    'random_state': [0]
}
gb_clf = GridSearchCV(GradientBoostingClassifier(), param_grid=param_gb, cv=3, scoring='roc_auc')
gb_clf.fit(X_train[labels], y_train)

In [ ]:
gb_clf.grid_scores_, gb_clf.best_params_ #max_depth: 5, learning_rate: 0.1

In [ ]:
gb_clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=0)
gb_clf.fit(X_train[labels], y_train)
result_gb = gb_clf.predict_proba(X_test[labels])[:,1]
#pd.DataFrame(result_gb).to_csv('results_gb.csv',index=True, index_label='test_id', header = ["is_female"])

In [ ]:
most_important = np.argsort(gb_clf.feature_importances_)[-10:]
np.asarray(labels)[most_important]

In [ ]:
# XGBOOST
param_xgb = {
    'max_depth': list(range(5,8,1)),
    'min_child_weight': list([1,2]),
}
xgb_clf = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=200, 
                                                 gamma=0.9, subsample=0.8, colsample_bytree=0.8,
                                                 objective= 'binary:logistic', nthread=5, seed=0),
                        param_grid = param_xgb, scoring='roc_auc',n_jobs=4,iid=False, cv=3)
xgb_clf.fit(X_train[labels], y_train)
xgb_clf.grid_scores_, xgb_clf.best_params_, xgb_clf.best_score_

In [ ]:
result_xgb = xgb_clf.predict_proba(X_test[labels])[:,1]
#pd.DataFrame(result_xgb).to_csv('results_xgb_1.csv',index=True, index_label='test_id', header = ["is_female"])

In [ ]:
most_important = np.argsort(xgb_clf.best_estimator_.feature_importances_)[-10:]
np.asarray(labels)[most_important]

### Random Forest Trees
Instead of combining several weak learners to one powerful model, random forest trees use several powerful learners to mitigate variance.

In [ ]:
param_rft={
    'n_estimators': [150],
    'max_depth': [20],
    'random_state': [0]
}
rft = GridSearchCV(estimator=RandomForestClassifier(),param_grid=param_rft, cv=3, scoring='roc_auc')
rft.fit(X_train[labels], y_train)
rft.grid_scores_, rft.best_params_, rft.best_score_

In [ ]:
result_rft = rft.predict_proba(X_test[labels])[:,1]
#pd.DataFrame(result_rft).to_csv('results_rft.csv',index=True, index_label='test_id', header = ["is_female"])

### Neural Networks
We use PyTorch as the framework for constructing neural network. For this binary classification problem, we use a single hidden layer with ReLU activation.

In [ ]:
transform=SelectKBest(chi2, k=500).fit(X_train, y_train)
support_index = np.array(np.where(transform.get_support()==True))[0]
labels = [X_train.columns.values.tolist()[i] for i in support_index]

M = np.shape(X_train[labels])[1] #number of features
H = 250 #number of hidden units
O = 2 #number of categories

class NNClassifier(nn.Module):

    def __init__(self, dim_in, dim_hidden, dim_out):
        super(NNClassifier, self).__init__()
        # 1 hidden layer, 2 output categories
        self.hidden = nn.Linear(dim_in, dim_hidden)
        self.out = nn.Linear(dim_hidden, dim_out)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.hidden(x)
        x = F.relu(x)
        x = self.out(x)
        x = self.softmax(x)
        return x

nn_clf = NNClassifier(M,H,O)

In [ ]:
E=200

nn_X = X_train[labels].values
nn_y = y_train.values
# divide into train and validation set
nn_X_train, nn_X_val, nn_y_train, nn_y_val = train_test_split(nn_X, nn_y, test_size=0.20)

input = Variable(torch.from_numpy(nn_X_train).float())
target = Variable(torch.from_numpy(nn_y_train), requires_grad=False)

optimizer = optim.SGD(nn_clf.parameters(),  lr = 0.01, momentum=0.9)
criterion = nn.NLLLoss()
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[350,500], gamma=0.1)
for epoch in range(E):
    scheduler.step()
    optimizer.zero_grad()   # zero the gradient buffers
    nn_clf.train(mode=True)
    output = nn_clf(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

print(loss.data)
# train
nn_prob_train = np.exp(output.data.numpy())
nn_roc_auc_train = roc_auc_score(y_true=target.data, y_score=nn_prob_train[:,1])
    
# validation
input_val = Variable(torch.from_numpy(nn_X_val).float())
target_val = Variable(torch.from_numpy(nn_y_val).long(), requires_grad=False)
nn_clf.eval()
output_val = nn_clf(input_val)
 
nn_prob_val = np.exp(output_val.data.numpy())
nn_roc_auc_val = roc_auc_score(y_true=target_val.data, y_score=nn_prob_val[:,1])

print("Validation ROC-AUC",nn_roc_auc_val)
print("Training ROC-AUC",nn_roc_auc_train)

In [ ]:
# predict
input_test = Variable(torch.from_numpy(X_test[labels].values).float())
nn_clf.eval()
output_test = nn_clf(input_test)
nn_prob_test = np.exp(output_test.data.numpy())[:,1]
#pd.DataFrame(nn_prob_test).to_csv('results_nn.csv',index=True, index_label='test_id', header = ["is_female"])

### VotingClassifier
We stack 3 models with a voting classifier. The voting classifier does not perform better than its best components alone.

In [ ]:
transformA=SelectKBest(chi2, k=140).fit(X_train, y_train)
support_indexA = np.array(np.where(transformA.get_support()==True))[0]
labelsA = [X_train.columns.values.tolist()[i] for i in support_indexA]

labels = labelsA

In [ ]:
param_stack = {
    'weights': [
        [0.2, 0.3, 0.5]
    ],
    'log_clf__C': [1],
    'xgb_clf__learning_rate': [0.03],
    'xgb_clf__n_estimators': [200],
    'xgb_clf__gamma': [0.9],
    'xgb_clf__subsample': [0.8],
    'xgb_clf__max_depth': [8],
    'xgb_clf__min_child_weight': [1],
    'xgb_clf__colsample_bytree': [0.8],
    'rft_clf__n_estimators': [200],
    'rft_clf__max_depth': [25]
}
estimators = [
    ('log_clf', LogisticRegression(random_state=0)), 
    ('rft_clf', RandomForestClassifier(random_state=0, n_jobs = -1)), 
    ('xgb_clf', XGBClassifier(objective='binary:logistic', nthread=5, seed=0))
]
stack_clf = GridSearchCV(VotingClassifier(estimators, voting='soft', n_jobs=1, flatten_transform=None), param_grid=param_stack, cv=3, scoring='roc_auc')

In [ ]:
stack_clf.fit(X_train[labels], y_train)

In [ ]:
stack_clf.grid_scores_, stack_clf.best_params_

In [ ]:
VotingClassifier(estimators=estimators).get_params()

In [ ]:
result_stack = stack_clf.predict_proba(X_test[labels])[:,1]
#pd.DataFrame(result_stack).to_csv('results_voting.csv',index=True, index_label='test_id', header = ["is_female"])